In [12]:
# https://pypi.org/project/selenium/
# https://selenium-python.readthedocs.io/api.html
!pip install selenium

In [13]:
pip install ipython

Note: you may need to restart the kernel to use updated packages.


pip install csv
pip install time 
pip install parsel
#pip install lxml
pip install bs4
pip install html5lib
pip install et_xmlfile

In [14]:
import pandas as pd
import lxml
import requests, time, random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [15]:
browser = webdriver.Chrome('chromedriver.exe')
browser.get('https://www.linkedin.com/uas/login')
file = open('config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]


elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

#elementID.submit()

# Slicing fonction Scrapping

In [16]:
def HTML_Sraping(link):
   
    browser.get(link)
    res=requests.get(link)
    SCROLL_PAUSE_TIME = 5
# Get scroll height
    y = 500
    y1 = 500
    # Get scroll height
    z = browser.execute_script("return document.body.scrollHeight")

        #pass
    for i in range(0,30):
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, "+str(y)+")")
        y += random.randint(300, 600) 
        time.sleep(0.1)
        # Wait to load page
        #time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height <= y:
            break
    for timer in range(0,30):
        browser.execute_script("window.scrollTo(0, "+str(z-y1)+")")
        y1 += random.randint(500, 600) 
        time.sleep(0.1)
        if z <= y1/2:
            break         
    try:
        browser.find_element_by_css_selector("button[data-control-name='skill_details']").click() 
        #print("clicked")
    except:
        pass
    src = browser.page_source
    #soup = BeautifulSoup(src, 'lxml')
    soup = BeautifulSoup(src,'lxml')
    #soup = src
    return soup    
    

In [17]:
def Json_Scraping_2(link):
    soup=HTML_Sraping(link)
    final_liste=[]
    info=dict()
    info["link"]=[link]
     ###########Personal info_name
    try:
        
        #info ={'link':'Vide','fullName':'Vide','profile_title':'Vide','location':'Vide','connectionsCount':'Vide','job_title':'Vide','company_name':'Vide','joining_date':'Vide','exp_duree':'Vide','college_name':'Vide','degree_name':'Vide','stream':'Vide','degree_year':'Vide','Language':'Vide'}
        name_div = soup.find('div', {'class': 'flex-1 mr5'})
        name_loc = name_div.find_all('ul')
        fullName = name_loc[0].find('li').get_text().strip()    
        
        info["fullName"]=[fullName]
    except:
        #print('no data')

        pass         
         ###########Personal info_Location   
    try:    
        name_div = soup.find('div', {'class': 'flex-1 mr5'})
        name_loc = name_div.find_all('ul')
        location = name_loc[1].find('li').get_text().strip()
        info["location"]=[location]
    except:
        #print('no data')

        pass 
         ###########Personal info_Profile_title
    try:
        name_div = soup.find('div', {'class': 'flex-1 mr5'})
        profile_title = name_div.find('h2').get_text().strip()
        info["profile_title"]=[profile_title]
    except:
        #print('no data')

        pass     

          ###########Personal info_ConnectionsCount
    try:
        name_div = soup.find('div', {'class': 'flex-1 mr5'})
        name_loc = name_div.find_all('ul')
        name_loc = name_div.find_all('ul')
        connection = name_loc[1].find_all('li')
        connectionsCount = connection[1].get_text().strip()

        info["connectionsCount"]=[connectionsCount]
    except:
        #print('no data')

        pass     

    ############## Experience
    try:
        exp_section = soup.find('section', {'id': 'experience-section'}).find('ul')
        job_title=[]
        jo=[]
        company_name=[]
        joining_date=[]
        exp_duree=[]
        for i in range(len(exp_section.find_all('div', {'class': "display-flex flex-column full-width"}))):
            a_tags=exp_section.find_all('div', {'class': "display-flex flex-column full-width"})[i].find('a')
            jo.append(a_tags.find('h3').get_text().strip())
            company_name .append( a_tags.find_all('p')[1].get_text().strip())
            joining_date .append(a_tags.find_all('h4')[0].find_all('span')[1].get_text().strip())
            exp_duree .append( a_tags.find_all('h4')[1].find_all('span')[1].get_text().strip())
            info["job_title"]=[jo]
            info["company_name"]=[company_name] 
            info["joining_date"]=[joining_date]
            info["exp_duree"]=[exp_duree]
    except:
        #print('no data')
        pass     

    #############Education( college_name)
    try:
        edu_section = soup.find('section', {'id': 'education-section'}).find('ul')
        college_name=[]

        for i in range(len(edu_section.find_all('div', {'class':"display-flex flex-column full-width"}))):
            edu=edu_section.find_all('div', {'class':"display-flex flex-column full-width"})[i]
            college_name.append(edu.find('h3').get_text().strip())
            info["college_name"]=[college_name]

    except:
        #print('no data')
        pass        
    #############Education( degree_name)
    try:
        edu_section = soup.find('section', {'id': 'education-section'}).find('ul')
    
        degree_name=[]
   
        for i in range(len(edu_section.find_all('div', {'class':"display-flex flex-column full-width"}))):
            edu=edu_section.find_all('div', {'class':"display-flex flex-column full-width"})[i]
            
            degree_name.append(edu.find('p', {'class': 'pv-entity__secondary-title pv-entity__degree-name t-14 t-black t-normal'}).find_all('span')[1].get_text().strip())
           
            info["degree_name"]=[degree_name]
            
    except:
        #print('no data')
        pass
    #############Education(stream)
    try:
        edu_section = soup.find('section', {'id': 'education-section'}).find('ul')
        
        stream =[]
        degree_year=[]
        for i in range(len(edu_section.find_all('div', {'class':"display-flex flex-column full-width"}))):
            edu=edu_section.find_all('div', {'class':"display-flex flex-column full-width"})[i]
            
            stream.append(edu.find('p', {'class': 'pv-entity__secondary-title pv-entity__fos t-14 t-black t-normal'}).find_all('span')[1].get_text().strip()) 
            info["stream"]=[stream]
            
    except:
        #print('no data')
        pass
    #############Education(degree_year)
    try:
        edu_section = soup.find('section', {'id': 'education-section'}).find('ul')
        degree_year=[]
        for i in range(len(edu_section.find_all('div', {'class':"display-flex flex-column full-width"}))):
            edu=edu_section.find_all('div', {'class':"display-flex flex-column full-width"})[i]
            degree_year.append( edu.find('p', {'class': 'pv-entity__dates t-14 t-black--light t-normal'}).find_all('span')[1].get_text().strip())
            info["degree_year"]=[degree_year]
    except:
        #print('no data')
        pass     
    #######Endorsements
    try:
        en=[]
        for i in range(len(soup.find_all('span', {'class':'custom-highlight t-bold'}))):
            en.append(soup.find_all('span', {'class':'custom-highlight t-bold'})[i].get_text().strip())
            info["endorsements"]=[en]  
    except:
        #print('no data')
        pass         

    ######Skills
    try:
        ss=[]
        for i in range(len(soup.find_all('div', {'class':'pv-skill-category-entity__skill-wrapper'}))):
            ss.append(soup.find_all('div', {'class':'pv-skill-category-entity__skill-wrapper'})[i].find('p',{'class':'pv-skill-category-entity__name tooltip-container'}).get_text().strip())
            info["skills"]=[ss]
    except:
        print('no data')

        pass
    ######Skills(Outils et technologie)
    try:
        ss=[]
        for i in range(len(soup.find_all('div', {'class':'pv-skill-category-entity__skill-wrapper'}))):
            ss.append(soup.find_all('div', {'class':'pv-skill-category-entity__skill-wrapper'})[i].find('p',{'class':'pv-skill-category-entity__name tooltip-container'}).get_text().strip())
            info["skills"]=[ss]
    except:
        #print('no data')

        pass     
 
    #####Accomplishments
    ####Language
    try:
        Accomp_section = soup.find('section', {'class': 'accordion-panel pv-profile-section pv-accomplishments-block languages ember-view'})        

        ll=[]
        for i in range(len(Accomp_section.find('ul').find_all('li'))):
            ll.append((Accomp_section.find('ul').find_all('li'))[i].get_text().strip())
            info["Language"]=[ll]
    except:
        #print('no data')
        pass 
    ######Projects
    try:
        pp=[]
        projets_section=soup.find('section', {'class': 'accordion-panel pv-profile-section pv-accomplishments-block projects ember-view'})        

        projets_section.find_all('li', {'class':"pv-accomplishments-block__summary-list-item"})
        for i in range(len(projets_section.find_all('li', {'class':"pv-accomplishments-block__summary-list-item"}))):
            pp.append(projets_section.find_all('li', {'class':"pv-accomplishments-block__summary-list-item"})[i].get_text().strip())
            info["project_num"]=[pp]
    except:
        #print('no data')
        pass     
    ######number of Projects
    try:
        projets_section=soup.find('section', {'class': 'accordion-panel pv-profile-section pv-accomplishments-block projects ember-view'})        

        for i in projets_section.find("h3").find('span', {'class':"visually-hidden"}).get_text().strip():        
            try:
                (int(i))
                info["nobre de projets"]=[int(i)]
                #print(int(i))
            except:
                pass
    except:
        #print('no data')
        pass     
 
    ######Publications and numbers
    try:
        publication_section = soup.find('div', {'id':'publications-expandable-content'})
        #publication=dict()
        p=[]
        #nombre=dict()
        nb=0
        
        for i in range(len(soup.find_all('div', {'id':'publications-expandable-content'}))):
            p.append(soup.find_all('div', {'id':'publications-expandable-content'})[i].find('li').get_text().strip())
            info["Publication"]=[p]
            nb=nb+1
            info["Pub_numbers"]=[nb]

    except:
        #print('no data')
        pass 
    ######Interests
    try:
        
        interests_section = soup.find('section', {'class':"pv-profile-section pv-interests-section artdeco-container-card ember-view"})
        interests_name=[]
        for i in range(len(interests_section.find_all('div', {'class':"pv-entity__summary-info ember-view"}))):
            inter=interests_section.find_all('div', {'class':"pv-entity__summary-info ember-view"})[i]
            interests_name.append(inter.find('span').get_text().strip())
            info["interests"]=[interests_name]
    except:
        print('no data')
        pass
    ######Certifications
    try:
        
        certif=[]
        for i in range(len(soup.find_all('h3', {'class':"t-16 t-bold"}))):
            certif.append(soup.find_all('h3', {'class':"t-16 t-bold"})[i].get_text().strip())
            info["certifications"]=[certif]
    except:
        #print('no data')
        pass       
    time.sleep(1)
    final_liste.append(info)
   
    return(final_liste)
    

In [18]:
urls=pd.read_csv("url_scrap.csv")

In [19]:
len(urls)

10000

In [20]:
def liste_final_scrapped(urls):
    liste=[]
    #for i in range(len(urls['url'])):
    for i in range(7501,8000):
        liste.append(Json_Scraping_2(urls['url'][i]+"/"))

    return liste


In [21]:
def Frame_final_scrapped(urls):    
    liste=liste_final_scrapped(urls)
    data_liste=[]
    for j in range(len(liste)):
        df=pd.DataFrame(liste[j])
        data_liste.append(df)
    return data_liste    

In [22]:
def data_final_scrapped(urls):
    data_liste=Frame_final_scrapped(urls)
    data_final=pd.concat(data_liste, axis=0, ignore_index=True)
    return data_final

In [23]:
data=data_final_scrapped(urls)

no data
no data


JavascriptException: Message: javascript error: Cannot read property 'scrollHeight' of null
  (Session info: chrome=80.0.3987.149)
